# Εργαστήριο 12
Όπως και στα προηγούμενα εργαστήρια συνεχίζουμε στο περιβάλλον του [online chisel bootcamp](https://mybinder.org/v2/gh/freechipsproject/chisel-bootcamp/master).

Πριν ξεκινήσετε, εκτελέστε τα επόμενα 2 κελιά:

In [ ]:
val path = System.getProperty("user.dir") + "/source/load-ivy.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chisel3.tester._
import chisel3.tester.RawTester.test
import dotvisualizer._

## Εντολές ανάγνωσης/εγγραφής (load/store) στη μνήμη δεδομένων
Στο εργαστήριο αυτό (και το επόμενο) θα κάνετε προσθήκες στα υπάρχοντα modules για να υλοποιήσετε τις εντολές **ανάγνωσης (load)** και **εγγραφής (store)** μιας λέξης στη μνήμη δεδομένων (data memory).

Η υλοποίηση θα γίνει σε 2 μέρη:

* Στο εργαστήριο αυτό θα τροποποιήσετε τα modules `FetchUnit` και `DataPath` για να προσθέσετε τη **μνήμη δεδομένων** στο σύστημα.

* Στο επόμενο εργαστήριο θα προσθέσετε λειτουργικότητα στα modules `DecodeUnit` και `Cpu` για την **εκτέλεση των εντολών load και store**.

Στις επόμενες ενότητες περιγράφεται η κωδικοποίηση και η αναμενόμενη λειτουργία των εντολών load και store.

### Εντολή load (ανάγνωση από τη μνήμη δεδομένων)

`load Dst,Rsrc2,Offset`

Η εντολή αυτή θα έχει την εξής λειτουργικότητα:

`Dst <- memory[address = Rsrc2 + Offset]`

όπου είναι:

* `Dst`: καταχωρητής προορισμού (3 bits)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (3 bits)
* `Offset`: σταθερά (ακέραιος με πρόσημο, 7 bits), προστίθεται στο περιεχόμενο του Rsrc2 για το σχηματισμό της διεύθυνσης μνήμης για ανάγνωση

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-13` | `bits 12-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- |
| `011` | `Offset[6:0]` | `Rsrc2` | `Dst` |


### Εντολή store (εγγραφή στη μνήμη δεδομένων)

`store Rsrc1,Rsrc2,Offset`

Η εντολή αυτή θα έχει την εξής λειτουργικότητα:

`memory[address = Rsrc2 + Offset] <- Rsrc1`

όπου είναι:

* `Rsrc1`: καταχωρητής εισόδου δεδομένων 1 (3 bits)
* `Rsrc2`: καταχωρητής εισόδου δεδομένων 2 (3 bits)
* `Offset`: σταθερά (ακέραιος με πρόσημο, 7 bits), προστίθεται στο περιεχόμενο του Rsrc2 για το σχηματισμό της διεύθυνσης μνήμης για ανάγνωση

Η κωδικοποίηση της εντολής αυτής έχει ως εξής:

| `bits 15-13` | `bits 12-9` | `bits 8-6` | `bits 5-3` | `bits 2-0` |
| --- | --- | --- | --- | --- |
| `111` | `Offset[6:3]` | `Rsrc2` | `Rsrc1` | `Offset[2:0]` |


## Υλοποίηση της μνήμης δεδομένων
Για την υλοποίηση της μνήμης δεδομένων θα χρησιμοποιηθεί η δομή `SyncReadMem` της Chisel:

~~~scala
// παράδειγμα κατασκευής μνήμης με 1024 λέξεις
// όπου κάθε λέξη περιέχει μη προσημασμένο αριθμό των 16 bits
val mem = SyncReadMem(1024,UInt(16.W))

// παράδειγμα ανάγνωσης μιας λέξης από τη διεύθυνση address
dataOut := mem.read(address)

// παράδειγμα εγγραφής της λέξης dataIn στη διεύθυνση address
mem.write(io.address,io.dataIn)
~~~

Με τη χρήση της `SyncReadMem`, και ανάλογα με την τεχνολογία κατασκευής, η μνήμη που προδιαγράφουμε υλοποιείται συνήθως ως στατική μνήμη (SRAM) μέσα στο τελικό chip. Το ταίριασμα των ζητούμενων χαρακτηριστικών της `SyncReadMem` με εκείνα που πραγματικά προσφέρει η τεχνολογία υλοποίησης είναι σύνθετη διαδικασία, δεν θα την καλύψουμε όμως στο εργαστήριο.

### Πολύ σημαντικό: η λειτουργικότητα της SyncReadMem
Η μνήμη που σχεδιάζεται με τη βοήθεια της `SyncReadMem` είναι **σύγχρονη**, λειτουργεί δηλαδή σε συγχρονισμό με το **σήμα ρολογιού** (clock) του συστήματος ως εξής:

1. Για την **εγγραφή**, πρώτα παρέχουμε τη *διεύθυνση εγγραφής* και τα *δεδομένα εγγραφής*. Στη συνέχεια η εγγραφή εκτελείται **στην έναρξη του επόμενου κύκλου ρολογιού**.

2. Για την **ανάγνωση**, πρώτα παρέχουμε τη *διεύθυνση ανάγνωσης*. Στη συνέχεια η ανάγνωση εκτελείται και **στην έναρξη του επόμενου κύκλου ρολογιού** εμφανίζονται στην έξοδο τα *δεδομένα ανάγνωσης*.

Το (2) είναι ιδιαίτερα σημαντικό γιατί **προϋποθέτει την εκτέλεση 2 κύκλων ρολογιού για την εντολή ανάγνωσης** (έναν κύκλο για να υπολογίσουμε τη διεύθυνση μνήμης και έναν δεύτερο κύκλο για να πάρουμε τα δεδομένα της ανάγνωσης), σε αντίθεση με όλες τις άλλες εντολές που ολοκληρώνονται σε έναν και μοναδικό κύκλο!

### Το module DataMemory
Στο επόμενο κελί εμφανίζεται το module `DataMemory` που περικλείει μια μνήμη τύπου `SyncReadMem`. Το μέγεθος της μνήμης σε λέξεις (`word_number`) και το εύρος κάθε λέξης (`word_width`) περιγράφονται παραμετρικά.

Το module `DataMemory` έχει τις εξής εισόδους και εξόδους:

* `address`: είσοδος εύρους log2Ceil(word_number) bits, η διεύθυνση εγγραφής ή ανάγνωσης.
* `dataIn`: είσοδος εύρους word_width bits, εδώ δίνονται τα δεδομένα εγγραφής.
* `wrEnable`: είσοδος 1 bit. Εκτελείται εγγραφή αν έχει την τιμή 1.
* `dataOut`: έξοδος εύρους word_width bits, εδώ εμφανίζονται τα δεδομένα ανάγνωσης.

Η συμπεριφορά της μνήμης όταν γίνεται ταυτόχρονη ανάγνωση και εγγραφή στην ίδια θέση μνήμης είναι *απροσδιόριστη*. Στη σχεδίαση του εργαστηρίου δεν έχουμε ποτέ ταυτόχρονη ανάγνωση και εγγραφή, συνεπώς δεν μας απασχολεί αυτό.

**Εκτελέστε** το επόμενο κελί:

In [ ]:
class DataMemory(word_number: Int,word_width: Int) extends Module {
  val io = IO(new Bundle {
    val address = Input(UInt(log2Ceil(word_number).W))
    val dataIn = Input(UInt(word_width.W))
    val wrEnable = Input(UInt(1.W))
    val dataOut = Output(UInt(word_width.W))        
  })
    
  val mem = SyncReadMem(word_number,UInt(word_width.W))
    
  io.dataOut := mem.read(io.address)
    
  when(io.wrEnable === 1.U) {
    mem.write(io.address,io.dataIn)    
  }
}

## Άσκηση 1: τροποποίηση του module FetchUnit

**Αντιγράψτε χωρίς αλλαγές** το module `InstructionMemory` από προηγούμενα εργαστήρια στο επόμενο κελί:

In [ ]:
class InstructionMemory(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {

  // ..συμπληρώστε..  
 
}

Στο επόμενο κελί **τροποποιήστε**  τον κώδικα του module `FetchUnit` από τα προηγούμενα εργαστήρια, έτσι ώστε να προστεθεί μια είσοδος του 1 bit με όνομα `freeze_pc`. Όταν η είσοδος `freeze_pc` είναι 1 δεν θα ενημερώνεται ο program counter (δεν θα προχωράει στην επόμενη τιμή). Με `freeze_pc` ίσο με 0, το `FetchUnit` θα λειτουργεί «κανονικά» (όπως στα προηγούμενα εργαστήρια).

Τι χρειάζεται το `freeze_pc`; Θυμηθείτε ότι έχουμε ως στόχο την προσθήκη μιας εντολής load που θα διαρκεί 2 κύκλους ρολογιού. Στον πρώτο κύκλο θα χρειαστεί να «παγώσουμε» τον pc, έτσι ώστε στον δεύτερο κύκλο να μην περάσουμε ακόμα στην επόμενη εντολή!

In [ ]:
class FetchUnit(addr_width: Int, instr_width: Int, content: Seq[UInt]) extends Module {

  // ..συμπληρώστε..  
 
}

Δοκιμάστε την ορθή λειτουργία του νέου `FetchUnit`:

In [ ]:
val instructions = Vector(33.U,1256.U,555.U)
test(new FetchUnit(8,16,instructions)) { c =>
  // δοκιμή με το freeze_pc ανενεργό  
  c.io.freeze_pc.poke(0.U)  
  // αύξηση pc κατά 1 και έλεγχος αντίστοιχης εξόδου  
  c.io.pc_sel.poke(0.U)
  for (i <- 0 until instructions.length) {
    c.io.instruction.expect(instructions(i))
    c.clock.step()  
  }
  // έλεγχος εξόδου με χρήση του branch pc  
  c.io.pc_sel.poke(1.U)
  c.io.branch_pc.poke(1.U)  // μεταπήδηση στη θέση 1
  c.clock.step()
  c.io.instruction.expect(instructions(1))
    
  // έλεγχος με το freeze_pc ενεργό
  c.io.freeze_pc.poke(1.U)
  c.clock.step()
  c.io.instruction.expect(instructions(1)) // πρέπει να παραμείνει στην προηγούμενη θέση (=1)  
}
println("SUCCESS!!")

## Άσκηση 2: τροποποίηση του module DataPath

**Αντιγράψτε χωρίς αλλαγές** τα modules `RegisterFile` και `Alu` στα επόμενα δύο κελιά:

In [ ]:
class RegisterFile(register_number: Int, register_width: Int) extends Module {

  // ..συμπληρώστε..  
 
}

In [ ]:
class Alu(n: Int) extends Module {

  // ..συμπληρώστε..  
 
}

Στη συνέχεια, **τροποποιήστε** το module `DataPath` σύμφωνα με το επόμενο σχήμα, εκτελώντας τα βήματα που περιγράφονται αμέσως μετά:

![datapath1.png](https://mixstef.github.io/courses/comparch/labimg/datapath2.png)

* **Βήμα 1ο**: το module `DataPath` θα έχει και τρίτη παράμετρο `memory_size` που θα περιγράφει το μέγεθος της μνήμης δεδομένων.
  ~~~scala
  class DataPath(register_number: Int, register_width: Int, memory_size: Int) extends Module {
  ~~~

* **Βήμα 2ο**: Προσθέσετε δύο νέες εισόδους του 1 bit η καθεμία, `write_en` και `result_sel` (βλ. στο σχήμα).
  * `write_en`: αν είναι 1, η μνήμη δεδομένων εκτελεί εγγραφή.
  * `result_sel`: επιλογή τελικού αποτελέσματος (0 = από alu, 1 = από μνήμη δεδομένων).
  
  
* **Βήμα 3ο**: Προσθέστε ένα instance του module `DataMemory`
  ~~~scala
  val dmem = Module(new DataMemory(memory_size,register_width))
  ~~~
  
  
* **Βήμα 4ο**: Ορίστε τον κόμβο `out_value` ως εξής:
  ~~~scala
  val out_value = Wire(UInt())
  ~~~
  και στη συνέχεια υλοποιήστε τις νέες συνδέσεις (φαίνονται με έντονο μαύρο στο σχήμα). Θυμηθείτε να αλλάξετε τη σύνδεση της εξόδου `io.results` (τώρα παίρνει τιμή από το `out_value`).
  
  **Σημείωση:** Στη σύνδεση του `dmem.io.address` με το `alu.io.out`, το `dmem.io.address` έχει λιγότερα bits από το `alu.io.out` και η Chisel, **στην παρούσα έκδοση**, θα φροντίσει να συνδέσει μόνο τα αντίστοιχα σήματα που υπάρχουν. Συνεπώς μπορείτε απλώς να γράψετε:
    ~~~scala
    dmem.io.address := alu.io.out
    ~~~


In [ ]:
class DataPath(register_number: Int, register_width: Int, memory_size: Int) extends Module {

  // ..συμπληρώστε..  
 
}

Δοκιμάστε τη λειτουργία του νέου module `DataPath` εκτελώντας το επόμενο κελί:

In [ ]:
test(new DataPath(8,16,1024)) { c =>
  // σήματα εισόδου για τις δοκιμαστικές "λειτουργίες" και τις αναμενόμενες εξόδους "results" και "zero"
  val cmdbits = List(Map("READ_SEL_A" -> 0.U,    // λειτουργία: r1 <- 33
                         "READ_SEL_B" -> 0.U,
                         "ALU_A_SEL" -> 0.U,
                         "IM" -> 33.U,
                         "SEL" -> "b01".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 1.U,
                         "WRITE_EN" -> 0.U,
                         "RESULT_SEL" -> 0.U,
                         "zero" -> 0.U,
                         "results" -> 33.U),
                     
                     Map("READ_SEL_A" -> 1.U,    // λειτουργία: r0 <- r1 - r1
                         "READ_SEL_B" -> 1.U,
                         "ALU_A_SEL" -> 1.U,
                         "IM" -> 0.U,
                         "SEL" -> "b11".U,
                         "SUB" -> 1.U,
                         "WRITE_SEL" -> 0.U,
                         "WRITE_EN" -> 0.U,
                         "RESULT_SEL" -> 0.U,
                         "zero" -> 1.U,
                         "results" -> 0.U),
                     
                     Map("READ_SEL_A" -> 1.U,    // λειτουργία: store mem[r0+1] <- r1 
                         "READ_SEL_B" -> 0.U,
                         "ALU_A_SEL" -> 0.U,
                         "IM" -> 1.U,
                         "SEL" -> "b11".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 0.U,
                         "WRITE_EN" -> 1.U,
                         "RESULT_SEL" -> 0.U,
                         "zero" -> 0.U,
                         "results" -> 1.U),  // results = διεύθυνση store (= r0+1 = 1)
                     
                     Map("READ_SEL_A" -> 1.U,    // λειτουργία: load r2 <- mem[r0+1] (1ος κύκλος)
                         "READ_SEL_B" -> 0.U,
                         "ALU_A_SEL" -> 0.U,
                         "IM" -> 1.U,
                         "SEL" -> "b11".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 0.U,
                         "WRITE_EN" -> 0.U,
                         "RESULT_SEL" -> 0.U,
                         "zero" -> 0.U,
                         "results" -> 1.U),   // results = διεύθυνση load (= r0+1 = 1)
                     
                     Map("READ_SEL_A" -> 0.U,    // λειτουργία: load r2 <- mem[r0+1] (2ος κύκλος)
                         "READ_SEL_B" -> 0.U,
                         "ALU_A_SEL" -> 1.U,
                         "IM" -> 0.U,
                         "SEL" -> "b00".U,
                         "SUB" -> 0.U,
                         "WRITE_SEL" -> 0.U,
                         "WRITE_EN" -> 0.U,
                         "RESULT_SEL" -> 1.U,  
                         "zero" -> 1.U,       // το zero παράγεται από το module alu, όχι από τη μνήμη
                         "results" -> 33.U) 
                    )
      
  for ((bits,i) <- cmdbits.zipWithIndex) {
    c.io.read_sel_a.poke(bits("READ_SEL_A"))
    c.io.read_sel_b.poke(bits("READ_SEL_B"))
    c.io.alu_a_sel.poke(bits("ALU_A_SEL"))
    c.io.im.poke(bits("IM"))
    c.io.sel.poke(bits("SEL"))
    c.io.sub.poke(bits("SUB"))
    c.io.write_sel.poke(bits("WRITE_SEL"))
    c.io.write_en.poke(bits("WRITE_EN"))
    c.io.result_sel.poke(bits("RESULT_SEL"))
    println(s"testing cycle $i, expecting zero=${bits("zero")},results=${bits("results")}")  
    c.io.zero.expect(bits("zero"))
    c.io.results.expect(bits("results"))
    c.clock.step()
  }  
}
println("SUCCESS!!")

## Τελειώνοντας...
Κατεβάσετε και πάρτε μαζί σας το σημερινό notebook, **θα το χρειαστείτε στο επόμενο εργαστήριο**.

**Δεν ανήκει στα παραδοτέα του εργαστηρίου.**